In [ ]:
import numpy as np
import time

from pollen_manipulation.reachy_2.api import Reachy2ManipulationAPI

from pollen_vision.camera_wrappers.depthai import SDKWrapper
from pollen_vision.camera_wrappers.depthai.utils import get_config_file_path
from pollen_vision.perception import Perception

import FramesViewer.utils as fv_utils
from reachy2_sdk import ReachySDK

import numpy as np
from pyquaternion import Quaternion
import ipywidgets as widgets
from IPython.display import display, clear_output

from scipy.spatial.transform import Rotation as R

T_world_cam = fv_utils.make_pose([0.049597, 0.009989, 0.038089], [0, 0, 0])
T_world_cam[:3, :3] = np.array([[0, 0, 1], [-1, 0, 0], [0, -1, 0]])
T_world_cam = fv_utils.rotateInSelf(T_world_cam, [-48, 0, 0])

cam = SDKWrapper(get_config_file_path("CONFIG_SR"), compute_depth=True)

K = cam.get_K()

# reachy = ReachySDK(host="192.168.1.42")
reachy = ReachySDK(host="localhost")
manipulation_api = Reachy2ManipulationAPI(reachy, T_world_cam, K)

perception = Perception(camera_wrapper=cam, T_world_cam=T_world_cam, freq=10.0)
perception.start(visualize=False)

time.sleep(1.0)
manipulation_api.turn_robot_on()
time.sleep(1.0)
manipulation_api.goto_rest_position(left=False)
manipulation_api.goto_rest_position(left=True)

In [ ]:
def get_object_info():
    object_info = [
        info for info in perception.get_objects_infos() if info["name"] == object_name and info["detection_score"] > 0.7
    ]

    if len(object_info) == 0:
        print(f"No {object_name} found")
        return []

    elif len(object_info) == 1:
        bottle_info = object_info[0]
        print(f"Found a {object_name}")
        return object_info[0]
    else:
        print(f"Found {len(object_info)} {object_name}")
        return []


def compute_goal_diff(fk_matrix, goal_pose):
    return np.linalg.norm(fk_matrix - selected_array)


def compute_l2_distance(fk_matrix, goal_pose):
    return np.linalg.norm(fk_matrix[:-1, 3] - goal_pose[:-1, 3])


def compute_rotation_distance(fk_matrix, goal_pose):
    q1 = Quaternion(matrix=fk_matrix[:3, :3])
    Q, R = np.linalg.qr(goal_pose[:3, :3])
    q2 = Quaternion(matrix=Q)
    return Quaternion.distance(q1, q2)


def produce_symetrical_poses(grasp_poses):
    symetrical_grasp_poses = []

    for grasp_pose in grasp_poses:
        symetrical_grasp_pose = np.eye(4)
        x, y, z = grasp_pose[:-1, 3]
        symetrical_grasp_pose[:-1, 3] = [x, -y, z]

        rotation_matrix = grasp_pose[:-1, :-1]
        roll, pitch, yaw = R.from_matrix(rotation_matrix).as_euler("xyz", degrees=True)
        symetrical_grasp_pose[:-1, :-1] = R.from_euler("xyz", [-roll, pitch, -yaw], degrees=True).as_matrix()
        symetrical_grasp_poses.append(symetrical_grasp_pose)

    return symetrical_grasp_poses


# Define the initial pose matrices for each arm used by goto_rest_position API method
right_start_pose = np.array(
    [
        [0.0, 0.0, -1.0, 0.20],
        [0.0, 1.0, 0.0, -0.24],
        [1.0, 0.0, 0.0, -0.23],
        [0.0, 0.0, 0.0, 1.0],
    ]
)
left_start_pose = np.array(
    [
        [0.0, 0.0, -1.0, 0.20],
        [0.0, 1.0, 0.0, 0.24],
        [1.0, 0.0, 0.0, -0.23],
        [0.0, 0.0, 0.0, 1.0],
    ]
)

In [ ]:
object_name = "mug"

perception.set_tracked_objects([object_name])

## Trigger grasping on fake robot
Takes the first reachable grasp pose in the list generated by graspnet and perform grasping.

In [ ]:
object_info = get_object_info()

if object_info == []:
    print("Can't try grasping")
else:
    manipulation_api.grasp_object(object_info, left=False)
    manipulation_api.goto_rest_position(left=False, open_gripper=False)

In [ ]:
from reachy2_sdk import ReachySDK
import numpy as np

reachy = ReachySDK(host='localhost')

In [ ]:
reachy.connect()

In [ ]:
grasp_pose_test = np.array(
    [[   -0.39428,     0.74524,    -0.53773,     0.42341],
       [   -0.91861,    -0.30272,     0.25401,    -0.13824],
       [   0.026514,     0.59412,     0.80394,    -0.23734],
       [          0,           0,           0,           1]]
)
grasp_pose_test[:3, :3] = np.eye(3)
grasp_pose_test

In [ ]:
reachy.l_arm.publish_grasp_poses([grasp_pose_test], [1.0])

In [ ]:
grasp_poses[0]

In [ ]:
manipulation_api.reachy.l_arm.goto_from_matrix(
    grasp_pose_test,
    duration=4.0,
    with_cartesian_interpolation=True,
    interpolation_frequency=120,
)

In [ ]:
manipulation_api.goto_rest_position(left=True)

## Get all reachable grasp poses candidates

First, get the list of all the reachable grasp poses. The rest position is also inserted as first position so that it is possible to put the arm back to the rest position with the slider when viewing the grasp pose candidates.

A list with symetrical grasp poses is also generated. The symmetry of a given grasp pose for an arm is such that the other arm should end up in the same position as if it was with a mirror. Having this was useful to check the symmetry of both arms.

In [ ]:
object_info = get_object_info()

if object_info == []:
    print("Can't get reachable grasp poses")
else:
    pose = object_info["pose"]
    rgb = object_info["rgb"]
    mask = object_info["mask"]
    depth = object_info["depth"]

    # Put this flag to False if you want to grasp with the right arm
    left = True

    if left:
        arm = manipulation_api.reachy.l_arm
        symetrical_arm = manipulation_api.reachy.r_arm
        start_pose = left_start_pose

    else:
        arm = manipulation_api.reachy.r_arm
        symetrical_arm = manipulation_api.reachy.l_arm
        start_pose = right_start_pose

    grasp_poses, scores, _, _ = manipulation_api.get_reachable_grasp_poses(rgb, depth, mask, left)
    # grasp_poses.insert(0, start_pose)
    # scores.insert(0, 1.0)

    symetrical_grasp_poses = produce_symetrical_poses(grasp_poses)

In [ ]:
reachy.l_arm.publish_grasp_poses(grasp_poses[-1:], scores[-1:])

In [ ]:
reachy.l_arm.publish_grasp_poses(grasp_poses[0:1], scores[0:1])

In [ ]:
reachy.connect()

In [ ]:
reachy.l_arm.publish_grasp_poses(grasp_poses, scores)

In [ ]:
R.from_matrix(grasp_poses[0][:3, :3]).as_quat()

In [ ]:
grasp_poses[0]

In [ ]:
manipulation_api.execute_grasp(grasp_poses[0], duration=2.0, left=True, use_cartesian_interpolation=True)

In [ ]:
manipulation_api.open_gripper(left=True)

In [ ]:
manipulation_api.goto_rest_position(left=False)

In [ ]:
from pollen_manipulation.utils import get_angle_dist

In [ ]:
mat = np.zeros((3, 3))

In [ ]:
mat[0,2] = -1
mat[1,1] = 1
mat[2,0] = 1

In [ ]:
get_angle_dist(np.eye(3), mat)

## Iterate over each reachable grasp pose candidate

Using the slider, iterate over each reachable grasp pose and send it to the fake robot to visualize what would be the movement of the arm in rviz.

Once the grasp pose reached, the l2 distance between the actual cartesian coordinates of the effector and the one from the grasp pose is computed. 

In [ ]:
slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(grasp_poses) - 1,
    step=1,
    description="Tableau:",
    orientation="horizontal",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="700px"),  # Ajustez la largeur ici
)

# Bouton pour exécuter l'action
button = widgets.Button(description="Send goal pose")

# Zone de sortie pour afficher les résultats
output = widgets.Output()


def on_slider_change(change):
    with output:
        clear_output()
        selected_index = change["new"]
        selected_array = grasp_poses[selected_index]
        print(f"Tableau {selected_index}: ")
        print(selected_array)
        print(f"Pose score: {scores[selected_index]}")

        goto_id = arm.goto_from_matrix(
            selected_array, with_cartesian_interpolation=True, interpolation_frequency=120, duration=2.0
        )

        # if goto_id != -1:
        #     while not manipulation_api.reachy.is_move_finished(goto_id):
        #         time.sleep(0.1)

        #     time.sleep(1.0)

        joints_pos = arm.get_joints_positions()
        fk = arm.forward_kinematics()

        l2_dist = compute_l2_distance(fk, selected_array)
        print(f"l2 dist: {l2_dist}")

        ## Uncomment the lines below to send symmetric grasp poses to the other arm

        # goto_id = symetrical_arm.goto_from_matrix(symetrical_grasp_poses[selected_index])
        # if goto_id != -1:
        #     while not manipulation_api.reachy.is_move_finished(goto_id):
        #         time.sleep(0.1)

        #     time.sleep(1.0)
        #     sym_fk = symetrical_arm.forward_kinematics()

        #     l2_dist = compute_l2_distance(sym_fk, symetrical_grasp_poses[selected_index])
        #     print(f'l2 dist for symetrical arm: {l2_dist}')
        #     print(symetrical_grasp_poses[selected_index])


# Associer la fonction de changement de valeur au slider
slider.observe(on_slider_change, names="value")

# Afficher les widgets
display(slider, output)

In [ ]:
arm.goto_from_matrix(left_start_pose, with_cartesian_interpolation=True, interpolation_frequency=120, duration=2.0)

In [ ]:
manipulation_api.goto_rest_position(left=True)
manipulation_api.goto_rest_position(left=False)